In [ ]:
def levenshtein(str1, str2):
    len_str1 = len(str1) + 1
    len_str2 = len(str2) + 1

    matrix = [[0 for n in range(len_str2)] for m in range(len_str1)]
    for i in range(len_str1):
        matrix[i][0] = i
    for j in range(len_str2):
        matrix[0][j] = j

    for i in range(1, len_str1):
        for j in range(1, len_str2):
            if str1[i - 1] == str2[j - 1]:
                cost = 0
            else:
                cost = 1
            matrix[i][j] = min(
                matrix[i - 1][j] + 1, matrix[i][j - 1] + 1, matrix[i - 1][j - 1] + cost
            )

    return matrix[len_str1 - 1][len_str2 - 1]


def levenshtein(str1, str2):
    if not str1:
        return len(str2)
    if not str2:
        return len(str1)
    if str1[0] == str2[0]:
        return levenshtein(str1[1:], str2[1:])
    return 1 + min(
        levenshtein(str1, str2[1:]),
        levenshtein(str1[1:], str2),
        levenshtein(str1[1:], str2[1:]),
    )

In [ ]:
class WordData:
    def __init__(self, language, id, word):
        self.language = language
        self.id = id
        self.word = word

    def __repr__(self):
        return f"WordData(language={self.language}, id={self.id}, word={self.word})"


# Inicialización de una instancia de la clase WordData
word_data_instance = WordData(language="es", id=1, word="casa")
print(word_data_instance)  # Output: WordData(language='es', id=1, word='casa')

# Obtener los atributos de la instancia
print(word_data_instance.language)  # Output: 'es'
print(word_data_instance.id)  # Output: 1
print(word_data_instance.word)  # Output: 'casa'

# Modificar los atributos de la instancia
word_data_instance.language = "en"
word_data_instance.id = 2
word_data_instance.word = "house"

print(word_data_instance)  # Output: WordData(language='en', id=2, word='house')

In [ ]:
class WordData:
    def __init__(self, language, id, word):
        self.language = language
        self.id = id
        self.word = word

    def __repr__(self):
        return f"WordData(language={self.language}, id={self.id}, word={self.word})"

    def nearestWord(self, dictionary):
        results = [
            (word_data, self.levenshtein(self.word, word_data.word))
            for word_data in dictionary
        ]
        return sorted(results, key=lambda x: x[1])

    @classmethod
    def nearestWord_cls(cls, word, dictionary):
        results = [
            (word_data, cls.levenshtein(word, word_data.word))
            for word_data in dictionary
        ]
        return sorted(results, key=lambda x: x[1])

    @staticmethod
    def levenshtein(str1, str2):
        m, n = len(str1), len(str2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]

        for i in range(m + 1):
            for j in range(n + 1):
                if i == 0:
                    dp[i][j] = j
                elif j == 0:
                    dp[i][j] = i
                elif str1[i - 1] == str2[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1]
                else:
                    dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

        return dp[m][n]


# Ejemplo de uso:
dictionary = [
    WordData("es", 1, "casa"),
    WordData("es", 2, "perro"),
    WordData("es", 3, "gato"),
]

word_instance = WordData("es", 4, "carro")

# Uso del método de instancia
print(word_instance.nearestWord(dictionary))

# Uso del método de clase
print(WordData.nearestWord_cls("carro", dictionary))

In [ ]:
import psycopg2
import json


class DictionaryDB:
    instance_count = 0
    connection = None

    def __init__(self):
        DictionaryDB.instance_count += 1
        self.cursor = None
        self.connection = DictionaryDB.getConnection()

    @classmethod
    def getConnection(cls):
        if cls.connection is None:
            with open("credentials.json", "r") as file:
                credentials = json.load(file)

            cls.connection = psycopg2.connect(
                database=credentials["database"],
                user=credentials["user"],
                password=credentials["password"],
                host=credentials["host"],
                port=credentials["port"],
            )
        return cls.connection

    def getCursor(self):
        if self.cursor is None:
            self.cursor = self.connection.cursor()
        return self.cursor

    def executeQuery(self, sql_query):
        cursor = self.getCursor()
        cursor.execute(sql_query)
        return cursor.fetchall()

    def getWord(self, word, language):
        cursor = self.getCursor()
        cursor.execute(
            "SELECT id, word, language FROM Words WHERE word = %s AND language = %s",
            (word, language),
        )

        result = cursor.fetchone()
        if result:
            return WordData(result[1], result[2], result[0])
        else:
            return WordData(None, None, None)

    def getWordByID(self, wordId):
        cursor = self.getCursor()
        cursor.execute("SELECT id, word, language FROM Words WHERE id = %s", (wordId,))

        result = cursor.fetchone()
        if result:
            return WordData(result[1], result[2], result[0])
        else:
            return WordData(None, None, None)

    def getLanguageDictionary(self, language):
        cursor = self.getCursor()
        cursor.execute("SELECT id, word FROM Words WHERE language = %s", (language,))
        return [WordData(word, language, id) for id, word in cursor.fetchall()]

    def getAllWords(self):
        cursor = self.getCursor()
        cursor.execute("SELECT id, word, language FROM Words")

        all_word_data = []
        for id, word, language in cursor.fetchall():
            all_word_data.append(WordData(word, language, id))

        return all_word_data

    def getAllErrors(self):
        cursor = self.getCursor()
        cursor.execute("SELECT * FROM getAllErrors()")

        return [
            ErrorData(
                word=row[1],
                times=row[3],
                id=row[0],
                wordData=WordData(word=row[6], language=row[4], id=row[5]),
            )
            for row in cursor.fetchall()
        ]

    def getErrorsByStr(self, word):
        cursor = self.getCursor()
        cursor.execute("SELECT * FROM getErrorsByStr(%s)", (word,))

        return [
            ErrorData(
                word=row[1],
                times=row[3],
                id=row[0],
                wordData=WordData(word=row[6], language=row[4], id=row[5]),
            )
            for row in cursor.fetchall()
        ]

    def getWordRelationships(self, word_data_instance):
        cursor = self.getCursor()
        cursor.execute(
            "SELECT * FROM getWordRelationships(%s)", (word_data_instance.id,)
        )

        word_relationship_data = []
        for relationship_id, word_id, word, language in cursor.fetchall():
            word_relationship_data.append(
                (relationship_id, WordData(word, language, word_id))
            )

        return word_relationship_data

    def addLanguage(self, language):
        cursor = self.getCursor()
        cursor.execute("SELECT addLanguage(%s)", (language,))
        self.connection.commit()

    def addWord(self, wordData):
        cursor = self.getCursor()
        cursor.execute("SELECT addWord(%s, %s)", (wordData.word, wordData.language))
        self.connection.commit()

    def addWordRelationship(self, wordData1, wordData2):
        cursor = self.getCursor()
        cursor.execute(
            "SELECT addWordRelationship(%s, %s)", (wordData1.id, wordData2.id)
        )
        self.connection.commit()

    def addError(self, errorData):
        cursor = self.getCursor()
        cursor.execute(
            "SELECT addError(%s, %s)", (errorData.word, errorData.wordData.id)
        )
        self.connection.commit()

    def closeCursor(self):
        if self.cursor is not None:
            self.cursor.close()
            self.cursor = None

    def __del__(self):
        DictionaryDB.instance_count -= 1
        # Aquí retiré los comentarios que estaban desactivando el cierre de la conexión
        """self.closeCursor()
        
        if DictionaryDB.instance_count == 0 and DictionaryDB.connection is not None:
            DictionaryDB.connection.close()
            DictionaryDB.connection = None
            print("La última instancia está siendo destruida, conexión cerrada.")"""


class WordData:
    def __init__(self, word, language=None, id=None):
        self.language = language
        self.id = id
        self.word = word

    def __repr__(self):
        return f"WordData(language={self.language}, id={self.id}, word={self.word})"

    def nearestWord(self, dictionary):
        results = [
            (word_data, self.levenshtein(self.word, word_data.word))
            for word_data in dictionary
        ]
        return sorted(results, key=lambda x: x[1])

    def nearestWordTop(self, dictionary):
        results = self.nearestWord(dictionary)
        if not results:
            return []

        min_distance = results[0][1]
        return [word_data for word_data in results if word_data[1] == min_distance]

    @classmethod
    def nearestWordStr(cls, word, dictionary):
        results = [
            (word_data, cls.levenshtein(word, word_data.word))
            for word_data in dictionary
        ]
        return sorted(results, key=lambda x: x[1])

    @classmethod
    def nearestWordTopStr(cls, word, dictionary):
        results = cls.nearestWordStr(word, dictionary)
        if not results:
            return []

        min_distance = results[0][1]
        return [word_data for word_data in results if word_data[1] == min_distance]

    @staticmethod
    def levenshtein(str1, str2):
        m, n = len(str1), len(str2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]

        for i in range(m + 1):
            for j in range(n + 1):
                if i == 0:
                    dp[i][j] = j
                elif j == 0:
                    dp[i][j] = i
                elif str1[i - 1] == str2[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1]
                else:
                    dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

        return dp[m][n]


class ErrorData:
    def __init__(self, word=None, times=None, id=None, wordData=None):
        self.word = word
        self.times = times
        self.id = id
        self.wordData = wordData

    def __repr__(self):
        return f"ErrorData(word={self.word}, times={self.times}, id={self.id}, word_data_instance={self.wordData})"


words = [
    "casa",
    "lago",
    "fruta",
    "flor",
    "jardín",
    "tiempo",
    "espejo",
    "color",
    "dulce",
    "rojo",
    "amarillo",
    "libro",
    "pluma",
    "coche",
    "cielo",
    "agua",
    "música",
    "reloj",
    "puerta",
    "ventana",
    "silla",
    "mesa",
    "lámpara",
    "zapato",
    "bolsa",
    "camino",
    "árbol",
    "piedra",
    "lluvia",
    "nieve",
    "pájaro",
    "animal",
    "verde",
    "blanco",
    "negro",
    "marrón",
    "azul",
    "violeta",
    "rosa",
    "naranja",
    "gris",
    "amarillo",
    "vino",
    "uva",
    "manzana",
    "pera",
    "naranja",
    "limón",
    "familia",
    "amigo",
    "sonrisa",
    "triste",
    "alegre",
    "contento",
    "feliz",
    "risa",
    "juego",
    "niño",
    "niña",
    "mujer",
    "hombre",
    "adulto",
    "joven",
    "anciano",
    "bebé",
    "abuelo",
    "abuela",
    "padre",
    "madre",
    "hermano",
    "hermana",
    "tío",
    "tía",
    "primo",
    "prima",
    "amigo",
    "amiga",
    "vecino",
    "vecina",
    "profesor",
    "profesora",
    "alumno",
    "alumna",
    "director",
    "directora",
    "presidente",
    "presidenta",
    "rey",
    "reina",
    "príncipe",
    "princesa",
    "doctor",
    "doctora",
    "enfermero",
    "enfermera",
    "policía",
    "bombero",
    "juez",
    "abogado",
    "ingeniero",
    "arquitecto",
    "artista",
    "músico",
    "cantante",
    "actor",
    "actriz",
    "escritor",
    "escritora",
    "poeta",
    "poetisa",
    "dibujante",
    "pintor",
    "pintora",
    "escultor",
    "escultora",
    "fotógrafo",
    "fotógrafa",
    "periodista",
    "reportero",
    "reportera",
    "presentador",
    "presentadora",
    "camarero",
    "camarera",
    "cocinero",
    "cocinera",
    "deportista",
    "futbolista",
    "basquetbolista",
    "nadador",
    "nadadora",
    "corredor",
    "corredora",
    "ciclista",
    "gimnasta",
    "bailarín",
    "bailarina",
    "peluquero",
    "peluquera",
    "jardinero",
    "jardinera",
    "limpiador",
    "limpiadora",
    "carpintero",
    "carpintera",
    "electricista",
    "fontanero",
    "mecánico",
    "panadero",
    "panadera",
    "pastelero",
    "pastelera",
    "agricultor",
    "agricultora",
    "veterinario",
    "veterinaria",
    "biólogo",
    "bióloga",
    "químico",
    "química",
    "físico",
    "física",
    "matemático",
    "matemática",
    "astrónomo",
    "astrónoma",
    "geólogo",
    "geóloga",
    "arqueólogo",
    "arqueóloga",
    "historiador",
    "historiadora",
    "filósofo",
    "filósofa",
    "sociólogo",
    "socióloga",
    "psicólogo",
    "psicóloga",
    "economista",
    "lenguaje",
    "cultura",
    "historia",
    "geografía",
    "ciencia",
    "matemáticas",
    "física",
    "química",
    "biología",
    "astronomía",
    "geología",
    "arqueología",
    "filosofía",
    "sociología",
    "psicología",
    "economía",
    "política",
    "religión",
    "arte",
    "literatura",
    "música",
    "pintura",
    "escultura",
    "fotografía",
    "cine",
    "teatro",
    "danza",
    "ballet",
    "ópera",
    "concierto",
    "orquesta",
    "coro",
    "banda",
    "guitarra",
    "piano",
    "violín",
    "flauta",
    "trompeta",
    "tambor",
    "batería",
    "armonica",
    "saxofón",
    "clarinete",
    "trombón",
    "oboe",
    "arpa",
    "acordeón",
    "gaita",
    "banjo",
    "mandolina",
    "ukelele",
    "violonchelo",
    "contrabajo",
    "gaita",
    "maraca",
    "castañuela",
    "triángulo",
    "xilófono",
    "vibráfono",
    "marimba",
    "timbal",
    "conga",
    "bongo",
    "djembé",
    "darbuka",
    "címbalo",
    "órgano",
    "clavicordio",
    "clavicémbalo",
    "celesta",
    "melódica",
    "kalimba",
    "balafón",
    "sintetizador",
    "theremin",
    "ondas",
    "martenot",
    "cristal",
    "armonio",
    "fliscorno",
    "corneta",
    "corno",
    "tuba",
    "eufonio",
    "sousafón",
    "bombardino",
    "fliscorno",
    "corneta",
    "piccolo",
    "fagot",
    "corne",
    "inglés",
    "cornamusa",
    "serpentón",
    "zurna",
    "suona",
    "sheng",
    "hulusi",
    "dizi",
    "xiao",
    "bawu",
    "paixiao",
    "shakuhachi",
    "sho",
    "hichiriki",
    "ryuteki",
    "nokan",
    "shinobue",
    "hotchiku",
    "quena",
    "siku",
    "zampoña",
    "quenacho",
    "quena",
    "quenacho",
    "ocarina",
    "siku",
    "zampoña",
    "ney",
    "kaval",
    "duduk",
    "frula",
    "kaval",
    "sopilka",
    "svirel",
    "fujara",
    "fue",
    "recorder",
    "flabiol",
    "gralla",
    "chalumeau",
    "shawn",
    "bladder",
    "pipe",
    "bagpipe",
    "tulum",
    "gaita",
    "mijwiz",
    "mizmar",
    "rhaita",
    "surna",
    "zurla",
    "sopila",
    "roženica",
    "taragot",
    "tárogató",
    "clarinete",
    "chalumeau",
    "tarogato",
    "tárogató",
    "clarinete",
    "chalumeau",
    "saxofón",
    "clarinete",
    "bajo",
    "clarinete",
    "contraalto",
    "clarinete",
    "contrabajo",
    "clarinete",
    "octocontralto",
    "clarinete",
    "octocontrabass",
    "clarinete",
    "octocontrabass",
    "sarrusófono",
    "sarrusófono",
    "contrabajo",
    "sarrusófono",
    "contrabajo",
    "sarrusófono",
    "contralto",
    "sarrusófono",
    "tenor",
    "sarrusófono",
    "barítono",
    "sarrusófono",
    "bajo",
    "sarrusófono",
    "soprano",
    "sarrusófono",
    "alto",
    "sarrusófono",
    "tenor",
    "sarrusófono",
    "barítono",
    "sarrusófono",
    "tárogató",
    "tarogato",
    "chalumeau",
    "sarrusófono",
    "tárogató",
    "tarogato",
    "chalumeau",
    "sarrusófono",
    "oboe",
    "corne",
    "inglés",
    "oboe",
    "d'amore",
    "oboe",
    "barítono",
    "heckelfón",
    "lupofón",
    "corno",
    "di",
    "caccia",
    "corno",
    "inglés",
    "corno",
    "di",
    "bassetto",
    "corno",
    "da",
    "caccia",
    "trompa",
    "natural",
    "trompa",
    "de",
    "varas",
    "trompa",
    "doble",
    "trompa",
    "triple",
    "trompa",
    "cuadruple",
    "trompa",
    "wagneriana",
    "corno",
    "da",
    "caccia",
    "corno",
    "di",
    "bassetto",
    "corno",
    "da",
    "caccia",
    "trompeta",
    "natural",
    "trompeta",
    "de",
    "varas",
    "trompeta",
    "piccolo",
    "trompeta",
    "de",
    "llaves",
    "trompeta",
    "soprano",
    "trompeta",
    "alto",
    "trompeta",
    "tenor",
    "trompeta",
    "baja",
    "trompeta",
    "contrabajo",
    "trompeta",
    "subcontrabajo",
    "corneta",
    "fliscorno",
    "corneta",
    "soprano",
    "corneta",
    "alto",
    "corneta",
    "tenor",
    "corneta",
    "baja",
    "fliscorno",
    "soprano",
    "fliscorno",
    "alto",
    "fliscorno",
    "tenor",
    "fliscorno",
    "barítono",
    "fliscorno",
    "bajo",
    "fliscorno",
    "contrabajo",
    "fliscorno",
    "subcontrabajo",
    "trombón",
    "de",
    "varas",
    "trombón",
    "alto",
    "trombón",
    "tenor",
    "trombón",
    "bajo",
    "trombón",
    "contrabajo",
    "trombón",
    "soprano",
    "sackbut",
    "serpentón",
    "oficleido",
    "alto",
    "oficleido",
    "tenor",
    "oficleido",
    "bajo",
    "oficleido",
    "contrabajo",
    "tuba",
    "wagneriana",
    "tuba",
    "tenor",
    "tuba",
    "baja",
    "tuba",
    "contrabajo",
    "tuba",
    "subcontrabajo",
    "sousafón",
    "bombardino",
    "fliscorno",
    "contrabajo",
    "fliscorno",
    "subcontrabajo",
    "trombón",
    "de",
    "varas",
    "trombón",
    "alto",
    "trombón",
    "tenor",
    "trombón",
    "bajo",
    "trombón",
    "contrabajo",
    "trombón",
    "soprano",
    "sackbut",
    "serpentón",
    "oficleido",
    "alto",
    "oficleido",
    "tenor",
    "oficleido",
    "bajo",
    "oficleido",
    "contrabajo",
    "tuba",
    "wagneriana",
    "tuba",
    "tenor",
    "tuba",
    "baja",
    "tuba",
    "contrabajo",
    "tuba",
    "subcontrabajo",
    "sousafón",
    "bombardino",
    "fliscorno",
    "contrabajo",
    "fliscorno",
    "subcontrabajo",
    "percusión",
    "timbal",
    "bombo",
    "caja",
    "pandereta",
    "tambor",
    "conga",
    "bongo",
    "djembe",
    "darbuka",
    "maraca",
    "guiro",
    "claves",
    "triángulo",
    "cencerro",
    "platillo",
    "gong",
    "campana",
    "xilófono",
    "vibráfono",
    "marimba",
    "glockenspiel",
    "carillón",
    "celesta",
    "piano",
    "órgano",
    "acordeón",
    "bandoneón",
    "armónica",
    "melódica",
    "kalimba",
    "mbira",
    "sanfona",
    "zanfona",
    "arpa",
    "lira",
    "cítara",
    "salterio",
    "bandurria",
    "laúd",
    "mandolina",
    "mandola",
    "mandocelo",
    "bouzouki",
    "balalaika",
    "domra",
    "baglama",
    "oud",
    "saz",
    "tanbur",
    "tar",
    "setar",
    "sitar",
    "veena",
    "sarod",
    "santur",
    "kanun",
    "qanun",
    "cimbalom",
    "hackbrett",
    "santoor",
    "yangqin",
    "zheng",
    "koto",
    "guzheng",
    "dan",
    "tranh",
    "gayageum",
    "geomungo",
    "kayagum",
    "komungo",
    "yanggeum",
    "ajaeng",
    "haegeum",
    "janggu",
    "buk",
    "jing",
    "k'kwaenggwari",
    "janggo"
    # ... (y así sucesivamente con el resto de las palabras)
]

# Eliminar duplicados convirtiendo la lista a un conjunto y luego de nuevo a una lista

#
try:
    dictionary_db = DictionaryDB()
    cur = dictionary_db.getCursor()
    cur.execute("SELECT restartDictionary();")
    dictionary_db.connection.commit()
    dictionary_db.addLanguage("es-MX")
    palabras_unicas = list(set(words))
    for palabra in palabras_unicas:
        # Asegúrate de que la palabra tenga una longitud válida antes de agregarla
        if len(palabra) <= 25:
            try:
                # Insertar la palabra en la tabla
                dictionary_db.addWord(WordData(palabra, "es-MX"))
            except Exception as e:
                print(f"No se pudo insertar la palabra '{palabra}': {e}")

    dictionary_db.addError(ErrorData("amongos", None, None, WordData(None, None, 1)))
except Exception as er:
    print(f"error: {er}")
    print("valio")
finally:
    del dictionary_db

In [68]:
import psycopg2
import json
from flask import Flask, request, jsonify
from flask_cors import CORS

class DictionaryDB:
    instance_count = 0
    connection = None

    def __init__(self):
        DictionaryDB.instance_count += 1
        self.cursor = None
        self.connection = DictionaryDB.getConnection()

    @classmethod
    def getConnection(cls):
        if cls.connection is None:
            with open("credentials.json", "r") as file:
                credentials = json.load(file)

            cls.connection = psycopg2.connect(
                database=credentials["database"],
                user=credentials["user"],
                password=credentials["password"],
                host=credentials["host"],
                port=credentials["port"],
            )
        return cls.connection

    def getCursor(self):
        if self.cursor is None:
            self.cursor = self.connection.cursor()
        return self.cursor

    def getWordStr(self, word1):
        if word1 is None:
            return WordData(None, None, None)

        cursor = self.getCursor()
        cursor.execute(
            "SELECT id, word, language FROM Words WHERE word = %s",
            (word1,),  # Agrega una coma para crear una tupla con un solo elemento
        )

        word_data = []
        for id, word, language in cursor.fetchall():
            word_data.append(WordData(word, language, id))
        return word_data
        
    def getWord(self, word, language):
        if word is None or language is None:
            return WordData(None, None, None)

        cursor = self.getCursor()
        cursor.execute(
            "SELECT id, word, language FROM Words WHERE word = %s AND language = %s",
            (word, language),
        )
        result = cursor.fetchone()
        if result:
            return WordData(result[1], result[2], result[0])
        else:
            return WordData(None, None, None)
        
    def getWordByID(self, wordId):
        if wordId is None:
            return WordData(None, None, None)

        cursor = self.getCursor()
        cursor.execute("SELECT id, word, language FROM Words WHERE id = %s", (wordId,))

        result = cursor.fetchone()
        if result:
            return WordData(result[1], result[2], result[0])
        else:
            return WordData(None, None, None)

    def getAllWords(self):
        cursor = self.getCursor()
        cursor.execute("SELECT id, word, language FROM Words")

        all_word_data = []
        for id, word, language in cursor.fetchall():
            all_word_data.append(WordData(word, language, id))

        return all_word_data

    def getLanguageDictionary(self, language):
        if language is None:
            return []

        cursor = self.getCursor()
        cursor.execute("SELECT id, word FROM Words WHERE language = %s", (language,))
        return [WordData(word, language, id) for id, word in cursor.fetchall()]

    def getWordRelationships(self, word_data_instance):
        if word_data_instance is None or word_data_instance.id is None:
            return []

        cursor = self.getCursor()
        cursor.execute(
            "SELECT * FROM getWordRelationships(%s)", (word_data_instance.id,)
        )

        word_relationship_data = []
        for relationship_id, word_id, word, language in cursor.fetchall():
            word_relationship_data.append(
                (relationship_id, WordData(word, language, word_id))
            )

        return word_relationship_data

    def getAllErrors(self):
        cursor = self.getCursor()
        cursor.execute("SELECT * FROM getAllErrors()")

        return [
            ErrorData(
                word=row[1],
                times=row[3],
                id=row[0],
                wordData=WordData(word=row[6], language=row[4], id=row[5]),
            )
            for row in cursor.fetchall()
        ]

    def getErrorsByStr(self, word):
        cursor = self.getCursor()
        cursor.execute("SELECT * FROM getErrorsByStr(%s)", (word,))

        return [
            ErrorData(
                word=row[1],
                times=row[3],
                id=row[0],
                wordData=WordData(word=row[6], language=row[4], id=row[5]),
            )
            for row in cursor.fetchall()
        ]

    def addLanguage(self, language):
        if language is None:
            return

        cursor = self.getCursor()
        cursor.execute("SELECT addLanguage(%s)", (language,))

        self.connection.commit()

    def addWord(self, wordData):
        if wordData is None or wordData.word is None or wordData.language is None:
            return

        cursor = self.getCursor()
        cursor.execute("SELECT addWord(%s, %s)", (wordData.word, wordData.language))

        self.connection.commit()

    def addWordRelationship(self, wordData1, wordData2):
        if (
            wordData1 is None
            or wordData2 is None
            or wordData1.id is None
            or wordData2.id is None
        ):
            return

        cursor = self.getCursor()
        cursor.execute(
            "SELECT addWordRelationship(%s, %s)", (wordData1.id, wordData2.id)
        )
        self.connection.commit()

    def addError(self, errorData):
        if (
            errorData is None
            or errorData.word is None
            or errorData.wordData is None
            or errorData.wordData.id is None
        ):
            return

        cursor = self.getCursor()
        cursor.execute(
            "SELECT addError(%s, %s)", (errorData.word, errorData.wordData.id)
        )
        self.connection.commit()

    def closeCursor(self):
        if self.cursor is not None:
            self.cursor.close()
            self.cursor = None

    def __del__(self):
        DictionaryDB.instance_count -= 1
        # Aquí retiré los comentarios que estaban desactivando el cierre de la conexión
        """self.closeCursor()
        if DictionaryDB.instance_count == 0 and DictionaryDB.connection is not None:
            DictionaryDB.connection.close()
            DictionaryDB.connection = None
            print("La última instancia está siendo destruida, conexión cerrada.")"""


class WordData:
    def __init__(self, word, language=None, id=None):
        self.language = language
        self.id = id
        self.word = word

    def __repr__(self):
        return f"WordData(language={self.language}, id={self.id}, word={self.word})"

    def nearestWord(self, dictionary):
        results = [
            (word_data, self.levenshtein(self.word, word_data.word))
            for word_data in dictionary
        ]
        return sorted(results, key=lambda x: x[1])

    def nearestWordTop(self, dictionary):
        results = self.nearestWord(dictionary)
        if not results:
            return []

        min_distance = results[0][1]
        return [word_data for word_data in results if word_data[1] == min_distance]

    @classmethod
    def nearestWordStr(cls, word, dictionary):
        results = [
            (word_data, cls.levenshtein(word, word_data.word))
            for word_data in dictionary
        ]
        return sorted(results, key=lambda x: x[1])

    @classmethod
    def nearestWordTopStr(cls, word, dictionary):
        results = cls.nearestWordStr(word, dictionary)
        if not results:
            return []

        min_distance = results[0][1]
        return [word_data for word_data in results if word_data[1] == min_distance]

    @staticmethod
    def levenshtein(str1, str2):
        m, n = len(str1), len(str2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]

        for i in range(m + 1):
            for j in range(n + 1):
                if i == 0:
                    dp[i][j] = j
                elif j == 0:
                    dp[i][j] = i
                elif str1[i - 1] == str2[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1]
                else:
                    dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

        return dp[m][n]
    
    def serialize(self):
        return {
            "wordId": self.id,
            "word": self.word,
            "language": self.language,
        }
    
    def isNone(self):
        if self.word == None and self.language == None and self.id == None:
            return True
        return False

class ErrorData:
    def __init__(self, word=None, times=None, id=None, wordData=None):
        self.word = word
        self.times = times
        self.id = id
        self.wordData = wordData

    def __repr__(self):
        return f"ErrorData(word={self.word}, times={self.times}, id={self.id}, word_data_instance={self.wordData})"

dictionary_db = DictionaryDB()

app = Flask(__name__)
CORS(app)



# Endpoint para buscar una palabra
@app.route('/lookForWord', methods=['POST'])
def look_for_word():
    request_data = request.get_json()
    word = request_data.get('word')
    language = request_data.get('language')


    possible_words = []
    word_data = WordData(None)

    # Realiza la búsqueda de la palabra en la base de datos
    if language == None or language == '':
        possible_words = dictionary_db.getWordStr(word)
        if len(possible_errors) == 1:
            response_data = {
                "word": [word.serialize() for word in possible_words]
                }
        else:
            response_data = {
                "suggestions": [word.serialize() for word in possible_words],
                "type": "options"
                }
    else:
        word_data = dictionary_db.getWord(word, language)
        response_data = {
            "word": [word_data.serialize()] if word_data else []
            }

    if len(possible_words) == 0 and word_data.isNone() == True:
        possible_errors = dictionary_db.getErrorsByStr(word)
        if len(possible_errors) > 0:
            response_data = {
                "suggestions": [error.wordData.serialize() for error in possible_errors],
                "type": "common_error",
                "most_repeated": possible_errors[0].times
                }
        else:
            response_data = {
                "suggestions": [],
                "type": "not_founded"
                }
            possible_errors = WordData.nearestWordTopStr(word,dictionary_db.getAllWords())
            if len(possible_errors) > 0:
                if possible_errors[0][1] <= 3:
                    response_data = {
                        "suggestions": [word[0].serialize() for word in possible_errors],
                        "type": "levenshtein_error",
                        "best_similarity": possible_errors[0][1]
                        }

    # Construye la respuesta en formato JSON
    
    return jsonify(response_data)

# Endpoint para agregar una palabra
@app.route('/addWord', methods=['POST'])
def add_word():
    request_data = request.get_json()
    word = request_data.get('word')
    language = request_data.get('language')
    
    # Agrega la palabra a la base de datos
    dictionary_db.addWord(WordData(word, language))
    
    # Construye la respuesta en formato JSON
    response_data = {"request": "addWord", "status": True}
    
    return jsonify(response_data)

# Endpoint para agregar una traducción
@app.route('/addTranslation', methods=['POST'])
def add_translation():
    request_data = request.get_json()
    main_word_id = request_data.get('mainWordId')
    word = request_data.get('word')
    language = request_data.get('language')
    
    # Agrega la traducción a la base de datos
    dictionary_db.addWordTranslation(main_word_id, word, language)
    
    # Construye la respuesta en formato JSON
    response_data = {"request": "addTranslation", "status": True}
    
    return jsonify(response_data)

# Endpoint para agregar un error
@app.route('/addError', methods=['POST'])
def add_word():
    request_data = request.get_json()
    word = request_data.get('word')
    wordId = request_data.get('wordId')
    
    # Agrega la palabra a la base de datos
    
    # Construye la respuesta en formato JSON
    response_data = {"request": "addError", "status": True}
    
    return jsonify(response_data)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)



# Obtienes todas las palabras de todas las lenguas
diccionario = dictionary_db.getAllWords()
"""
print(diccionario)
diccionario = dictionary_db.getLanguageDictionary("es-MX")
print(diccionario)
"""

print("se han obtenido todas las palabras")

palabra = WordData("carro")

results = palabra.nearestWordTop(diccionario)
print("se han procesado todas las palabras")

for res in results:
    print(
        f"La distancia de Levenshtein entre '{palabra.word}' y '{res[0].word}' es: {res[1]}"
    )
print(
    dictionary_db.addWordRelationship(
        dictionary_db.getWord("corneta", "es-MX"),
        dictionary_db.getWord("coro", "es-MX"),
    )
)
#print(dictionary_db.getWordRelationships(dictionary_db.getWord("corneta", "es-MX")))

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.172:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Sep/2023 02:51:56] "OPTIONS /lookForWord HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2023 02:51:57] "POST /lookForWord HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2023 02:52:46] "OPTIONS /lookForWord HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2023 02:52:46] "POST /lookForWord HTTP/1.1" 200 -


se han obtenido todas las palabras
se han procesado todas las palabras
La distancia de Levenshtein entre 'carro' y 'corno' es: 2
La distancia de Levenshtein entre 'carro' y 'coro' es: 2
None


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Ruta para recibir datos JSON mediante POST
@app.route('/api/receive_json', methods=['POST'])
def receive_json():
    data = request.get_json()  # Obtener los datos JSON de la solicitud

    # Procesar los datos recibidos (aquí puedes realizar las operaciones que necesites)
    result = {"message": "Datos recibidos correctamente", "data": data}

    return jsonify(result)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


In [ ]:
!pip install flask-cors

